In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import datetime
import yfinance as yf
import pyodbc
import FinanceLib as fl
import time
import os
import sys
from datetime import timedelta, date
import requests
import bs4 as bs
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

D:\Projects\finance-venv\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [2]:
import psycopg2
def get_conn_to_pg():
    return psycopg2.connect(
       database="postgres", user='postgres', password=os.environ['PG_PWD'], host='127.0.0.1', port= '5432'
    )

In [3]:
def save_sp500_tickers():
    conn = get_conn_to_pg()
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    cursor = conn.cursor()
    
    for row in table.findAll('tr')[1:]:
        ticker = [row.findAll('td')[0].text.replace('\n', ''), row.findAll('td')[1].text.replace("'", "''"),
                  row.findAll('td')[2].text, row.findAll('td')[3].text]
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    for ticker in tickers:
        print(ticker)
        stock = ticker[0]
        security =  ticker[1]
        sector = ticker[2]
        sub_industry = ticker[3]
        
        query = (f"CALL finance.pLoadStockList('{stock}', '{security}' , '{sector}', '{sub_industry}');")
        print(query)
        cursor.execute(query)
    conn.commit();    
    conn.close()
        
    return tickers

In [5]:
save_sp500_tickers()

['MMM', '3M', 'Industrials', 'Industrial Conglomerates']
CALL finance.pLoadStockList('MMM', '3M' , 'Industrials', 'Industrial Conglomerates');
['AOS', 'A. O. Smith', 'Industrials', 'Building Products']
CALL finance.pLoadStockList('AOS', 'A. O. Smith' , 'Industrials', 'Building Products');
['ABT', 'Abbott', 'Health Care', 'Health Care Equipment']
CALL finance.pLoadStockList('ABT', 'Abbott' , 'Health Care', 'Health Care Equipment');
['ABBV', 'AbbVie', 'Health Care', 'Pharmaceuticals']
CALL finance.pLoadStockList('ABBV', 'AbbVie' , 'Health Care', 'Pharmaceuticals');
['ACN', 'Accenture', 'Information Technology', 'IT Consulting & Other Services']
CALL finance.pLoadStockList('ACN', 'Accenture' , 'Information Technology', 'IT Consulting & Other Services');
['ATVI', 'Activision Blizzard', 'Communication Services', 'Interactive Home Entertainment']
CALL finance.pLoadStockList('ATVI', 'Activision Blizzard' , 'Communication Services', 'Interactive Home Entertainment');
['ADM', 'ADM', 'Consumer S

[['MMM', '3M', 'Industrials', 'Industrial Conglomerates'],
 ['AOS', 'A. O. Smith', 'Industrials', 'Building Products'],
 ['ABT', 'Abbott', 'Health Care', 'Health Care Equipment'],
 ['ABBV', 'AbbVie', 'Health Care', 'Pharmaceuticals'],
 ['ACN',
  'Accenture',
  'Information Technology',
  'IT Consulting & Other Services'],
 ['ATVI',
  'Activision Blizzard',
  'Communication Services',
  'Interactive Home Entertainment'],
 ['ADM', 'ADM', 'Consumer Staples', 'Agricultural Products & Services'],
 ['ADBE', 'Adobe Inc.', 'Information Technology', 'Application Software'],
 ['ADP', 'ADP', 'Industrials', 'Human Resource & Employment Services'],
 ['AAP', 'Advance Auto Parts', 'Consumer Discretionary', 'Automotive Retail'],
 ['AES',
  'AES Corporation',
  'Utilities',
  'Independent Power Producers & Energy Traders'],
 ['AFL', 'Aflac', 'Financials', 'Life & Health Insurance'],
 ['A', 'Agilent Technologies', 'Health Care', 'Health Care Equipment'],
 ['APD', 'Air Products and Chemicals', 'Materials

In [22]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = []
    ticker.append(row.findAll('td')[0].text.replace('\n', ''))
    ticker.append(row.findAll('td')[1].text.replace("'", "''"))
    ticker.append(row.findAll('td')[2].text)
    ticker.append(row.findAll('td')[3].text)
    tickers.append(ticker)

In [28]:
tickers[0]

['MMM', '3M', 'Industrials', 'Industrial Conglomerates']

In [23]:
for ticker in tickers[0]:
    print(ticker)

MMM
3M
Industrials
Industrial Conglomerates


In [23]:
def save_stock_quote_to_db(stock, date_from, date_to):
    conn = get_conn_to_pg()
    try:
        data = yf.download(stock, date_from, date_to)
        df_to_lst = data.reset_index().values.tolist()
    except Exception as e:
        print(stock + ' Ошибка при получении данных из источника: ' + str(e)) 
        result = 1
    cursor = conn.cursor()
    try:
        for row in df_to_lst:
            date = row[0]
            open = row[1]
            high = row[2]
            low = row[3]
            close = row[4]
            adj_close = row[5]
            volume = row[6]
            query = (f"CALL finance.pLoadQuote('{date}', '{stock}', {open}, {high}, {low}, {close}, {adj_close}, {volume})")
            print(query)
            cursor.execute(query)
        conn.commit()
        conn.close()
        result = 0
    except Exception as e:
        print(stock + ' Ошибка при сохранении в базу: ' + str(e)) 
        result = 1
    return result

In [24]:
save_stock_quote_to_db("AAPL", "2023-01-01", "2023-06-01")

[*********************100%***********************]  1 of 1 completed
CALL finance.pLoadQuote('2023-01-03 00:00:00', 'AAPL', 130.27999877929688, 130.89999389648438, 124.16999816894531, 125.06999969482422, 124.70683288574219, 112117500)
CALL finance.pLoadQuote('2023-01-04 00:00:00', 'AAPL', 126.88999938964844, 128.66000366210938, 125.08000183105469, 126.36000061035156, 125.99309539794922, 89113600)
CALL finance.pLoadQuote('2023-01-05 00:00:00', 'AAPL', 127.12999725341797, 127.7699966430664, 124.76000213623047, 125.0199966430664, 124.65697479248047, 80962700)
CALL finance.pLoadQuote('2023-01-06 00:00:00', 'AAPL', 126.01000213623047, 130.2899932861328, 124.88999938964844, 129.6199951171875, 129.24362182617188, 87754700)
CALL finance.pLoadQuote('2023-01-09 00:00:00', 'AAPL', 130.47000122070312, 133.41000366210938, 129.88999938964844, 130.14999389648438, 129.77207946777344, 70790800)
CALL finance.pLoadQuote('2023-01-10 00:00:00', 'AAPL', 130.25999450683594, 131.25999450683594, 128.1199951171

0

In [30]:
def get_stock_list_from_db(market=None):
    conn = get_conn_to_pg()
    market_str = f"'{market}'" if market else'NULL'
    query = f"select * from finance.pGetStockList({market_str})"
    df = pd.read_sql(query, conn)
    return df

In [31]:
df = get_stock_list_from_db(market=None)

select * from finance.pGetStockList(NULL)


C:\Users\user\AppData\Roaming\MobaXterm\slash\tmp\ipykernel_23092\2012977843.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [32]:
df.head()

,stock,security,sector,subindustry,market,loaddt
0,MMM,3M,Industrials,Industrial Conglomerates,None,2023-06-06 23:20:27.298103
1,AOS,A. O. Smith,Industrials,Building Products,None,2023-06-06 23:20:27.298103
2,ABT,Abbott,Health Care,Health Care Equipment,None,2023-06-06 23:20:27.298103
3,ABBV,AbbVie,Health Care,Pharmaceuticals,None,2023-06-06 23:20:27.298103
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,None,2023-06-06 23:20:27.298103


In [1]:
import Financelib as fl

ModuleNotFoundError: No module named 'Financelib'

In [ ]:
fl.save_stock_quote_to_db(stock, date_from, date_to)

In [2]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import FinanceLib as fl

D:\Projects\finance-venv\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [6]:
# os.environ['PG_PWD']

In [8]:
fl.save_stock_quote_to_db("AAPL", "2023-01-01", "2023-06-11")

[*********************100%***********************]  1 of 1 completed


0